In [3]:
from tqdm import tqdm, trange
import pandas as pd

In [7]:
from searchapp.models import Entity, Label

In [8]:
df_nodes = pd.read_csv("../data/kg_nodes_id.csv", low_memory=False)

In [9]:
df_nodes

,id:ID,x_id,:LABEL,name,source,original_name
0,NCBI_9796_PHYHIP,9796,Gene/Protein,phyhip,NCBI,PHYHIP
1,NCBI_7918_GPANK1,7918,Gene/Protein,gpank1,NCBI,GPANK1
2,NCBI_8233_ZRSR2,8233,Gene/Protein,zrsr2,NCBI,ZRSR2
3,NCBI_4899_NRF1,4899,Gene/Protein,nrf1,NCBI,NRF1
4,NCBI_5297_PI4KA,5297,Gene/Protein,pi4ka,NCBI,PI4KA
...,...,...,...,...,...,...
129370,REACTOME_R-HSA-936837_Ion_transport_by_P-type_...,R-HSA-936837,Pathway,ion transport by p-type atpases,REACTOME,Ion transport by P-type ATPases
129371,REACTOME_R-HSA-997272_Inhibition__of_voltage_g...,R-HSA-997272,Pathway,inhibition of voltage gated ca2+ channels via...,REACTOME,Inhibition of voltage gated Ca2+ channels via...
129372,UBERON_1062_anatomical_entity,1062,Anatomy,anatomical entity,UBERON,anatomical entity
129373,UBERON_468_multi-cellular_organism,468,Anatomy,multi-cellular organism,UBERON,multi-cellular organism


In [11]:
df_nodes.drop_duplicates(subset=['id:ID'])

,id:ID,x_id,:LABEL,name,source,original_name
0,NCBI_9796_PHYHIP,9796,Gene/Protein,phyhip,NCBI,PHYHIP
1,NCBI_7918_GPANK1,7918,Gene/Protein,gpank1,NCBI,GPANK1
2,NCBI_8233_ZRSR2,8233,Gene/Protein,zrsr2,NCBI,ZRSR2
3,NCBI_4899_NRF1,4899,Gene/Protein,nrf1,NCBI,NRF1
4,NCBI_5297_PI4KA,5297,Gene/Protein,pi4ka,NCBI,PI4KA
...,...,...,...,...,...,...
129370,REACTOME_R-HSA-936837_Ion_transport_by_P-type_...,R-HSA-936837,Pathway,ion transport by p-type atpases,REACTOME,Ion transport by P-type ATPases
129371,REACTOME_R-HSA-997272_Inhibition__of_voltage_g...,R-HSA-997272,Pathway,inhibition of voltage gated ca2+ channels via...,REACTOME,Inhibition of voltage gated Ca2+ channels via...
129372,UBERON_1062_anatomical_entity,1062,Anatomy,anatomical entity,UBERON,anatomical entity
129373,UBERON_468_multi-cellular_organism,468,Anatomy,multi-cellular organism,UBERON,multi-cellular organism


# Import Label

In [6]:
label_data_list = df_nodes.drop_duplicates(subset=[':LABEL']).to_dict("records")
print(len(label_data_list))
label_data_list[:2]

10


[{'id:ID': 'NCBI_9796_PHYHIP',
  ':LABEL': 'Gene/Protein',
  'name': 'phyhip',
  'source': 'NCBI',
  'original_name': 'PHYHIP'},
 {'id:ID': 'DrugBank_DB09130_Copper',
  ':LABEL': 'Drug',
  'name': 'copper',
  'source': 'DrugBank',
  'original_name': 'Copper'}]

In [17]:
label_bulk_data = [Label(name=row.get(":LABEL")) for row in label_data_list]
label_bulk_data

[<Label: Gene/Protein>,
 <Label: Drug>,
 <Label: Effect/Phenotype>,
 <Label: Disease>,
 <Label: Biological_Process>,
 <Label: Molecular_Function>,
 <Label: Cellular_Component>,
 <Label: Exposure>,
 <Label: Pathway>,
 <Label: Anatomy>]

In [19]:
Label.objects.bulk_create(label_bulk_data)

[<Label: Gene/Protein>,
 <Label: Drug>,
 <Label: Effect/Phenotype>,
 <Label: Disease>,
 <Label: Biological_Process>,
 <Label: Molecular_Function>,
 <Label: Cellular_Component>,
 <Label: Exposure>,
 <Label: Pathway>,
 <Label: Anatomy>]

# Import Nodes

In [12]:
step = 1_000

In [13]:
nodes_list = df_nodes.to_dict("records")

In [14]:
nodes_list[:3]

[{'id:ID': 'NCBI_9796_PHYHIP',
  'x_id': '9796',
  ':LABEL': 'Gene/Protein',
  'name': 'phyhip',
  'source': 'NCBI',
  'original_name': 'PHYHIP'},
 {'id:ID': 'NCBI_7918_GPANK1',
  'x_id': '7918',
  ':LABEL': 'Gene/Protein',
  'name': 'gpank1',
  'source': 'NCBI',
  'original_name': 'GPANK1'},
 {'id:ID': 'NCBI_8233_ZRSR2',
  'x_id': '8233',
  ':LABEL': 'Gene/Protein',
  'name': 'zrsr2',
  'source': 'NCBI',
  'original_name': 'ZRSR2'}]

In [15]:
data_len = len(df_nodes)
data_len

129375

In [16]:
for i in tqdm(range(0, data_len, step), desc="import nodes to django: Entity"):
    bulk_data = []
    for row in nodes_list[i: i+step]:
        label = Label.objects.get(name=row.get(":LABEL"))
        entity = Entity(
            label=label,
            id=row.get("id:ID"),
            name=row.get("original_name"),
            attribute={
                "source": row.get("source", None),
                "label": row.get(":LABEL"),
                "node_id": row.get("x_id")
            }
        )
        bulk_data.append(entity)
    Entity.objects.bulk_create(bulk_data)

import nodes to django: Entity:   0%|          | 0/130 [00:01<?, ?it/s]


IntegrityError: duplicate key value violates unique constraint "searchapp_entity_pkey"
DETAIL:  Key (id)=(NCBI_9796_PHYHIP) already exists.


In [21]:
# Label.objects.all().delete()

In [11]:
Entity.objects.all().delete()

IntegrityError: update or delete on table "searchapp_entity" violates foreign key constraint "searchapp_link_entity_id_a84cfad5_fk_searchapp_entity_id" on table "searchapp_link"
DETAIL:  Key (id)=(CTD_C024746_tobacco_tar) is still referenced from table "searchapp_link".
